<a href="https://colab.research.google.com/github/sasiking2003-boop/Trading/blob/main/Stock_Market_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import datetime
import os
import time
from zipfile import ZipFile
from io import BytesIO

class MarketDataDownloader:
    def __init__(self):
        self.download_dir = "market_data"
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

        # NSE requires valid User-Agent and Session Cookies
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive'
        }
        self.session = requests.Session()
        self.session.headers.update(self.headers)

    def _initialize_nse_cookies(self):
        """
        NSE blocks requests without valid cookies. We visit the homepage first
        to get these cookies before attempting the download.
        """
        try:
            print("Initializing NSE session (fetching cookies)...")
            response = self.session.get("https://www.nseindia.com", timeout=10)
            if response.status_code == 200:
                print("Session established successfully.")
            else:
                print(f"Failed to initialize NSE session. Status: {response.status_code}")
        except Exception as e:
            print(f"Error establishing session: {e}")

    def download_nse_bhavcopy(self, date_obj):
        """
        Downloads NSE Cash Market (CM) Bhavcopy.
        New Format (post-July 2024): BhavCopy_NSE_CM_0_0_0_YYYYMMDD_F_0000.csv.zip
        """
        self._initialize_nse_cookies()

        # Format: YYYYMMDD (e.g., 20251126)
        date_str = date_obj.strftime("%Y%m%d")

        # New URL Pattern for 2025
        filename = f"BhavCopy_NSE_CM_0_0_0_{date_str}_F_0000.csv.zip"
        url = f"https://nsearchives.nseindia.com/content/cm/{filename}"

        print(f"\n[NSE] Attempting to download: {filename}")
        try:
            response = self.session.get(url, timeout=15)

            if response.status_code == 200:
                print("[NSE] Download successful. Extracting...")
                with ZipFile(BytesIO(response.content)) as z:
                    z.extractall(self.download_dir)
                print(f"[NSE] Saved to folder: {self.download_dir}")
                return True
            elif response.status_code == 404:
                print("[NSE] File not found. The market might be closed or data not updated yet (Try after 6:30 PM IST).")
            else:
                print(f"[NSE] Failed with status code: {response.status_code}")
        except Exception as e:
            print(f"[NSE] Error: {e}")

        return False

    def download_bse_bhavcopy(self, date_obj):
        """
        Downloads BSE Equity Bhavcopy.
        Format: EQDDMMYY_CSV.ZIP (e.g., EQ261125_CSV.ZIP)
        """
        # Format: DDMMYY (e.g., 261125)
        date_str = date_obj.strftime("%d%m%y")
        filename = f"EQ{date_str}_CSV.ZIP"

        # BSE URL Pattern
        url = f"https://www.bseindia.com/download/BhavCopy/Equity/{filename}"

        print(f"\n[BSE] Attempting to download: {filename}")
        try:
            # BSE usually doesn't require strict cookies, but headers help
            response = requests.get(url, headers=self.headers, timeout=15)

            if response.status_code == 200:
                print("[BSE] Download successful. Extracting...")
                with ZipFile(BytesIO(response.content)) as z:
                    z.extractall(self.download_dir)
                print(f"[BSE] Saved to folder: {self.download_dir}")
                return True
            elif response.status_code == 404:
                print("[BSE] File not found. Market might be closed or report not generated yet.")
            else:
                print(f"[BSE] Failed with status code: {response.status_code}")
        except Exception as e:
            print(f"[BSE] Error: {e}")

        return False

def get_latest_trading_date():
    """
    Returns the most recent valid trading date based on current time.
    If it's before 6:00 PM (18:00), it assumes today's report isn't ready
    and returns yesterday (or last Friday).
    """
    now = datetime.datetime.now()
    today_date = now.date()

    # If current time is before 6:00 PM, the report is likely not generated yet.
    # So we look for the previous day.
    if now.hour < 18:
        print("Current time is before 6:00 PM. Fetching PREVIOUS trading day's data.")
        target_date = today_date - datetime.timedelta(days=1)
    else:
        print("Current time is after 6:00 PM. Fetching TODAY'S data.")
        target_date = today_date

    # Adjust for weekends
    # If target is Sunday (6), go back to Friday (2 days)
    if target_date.weekday() == 6:
        target_date -= datetime.timedelta(days=2)
    # If target is Saturday (5), go back to Friday (1 day)
    elif target_date.weekday() == 5:
        target_date -= datetime.timedelta(days=1)

    return target_date

if __name__ == "__main__":
    downloader = MarketDataDownloader()

    # Calculate the correct date to download
    target_date = get_latest_trading_date()

    print(f"--- Starting Download for Date: {target_date.strftime('%d-%b-%Y')} ---")

    # Download NSE
    downloader.download_nse_bhavcopy(target_date)

    # Download BSE
    downloader.download_bse_bhavcopy(target_date)

    print("\nProcessing complete.")